In [34]:
import graph_tool.all as gt
# import matplotlib
import csv, os
# import pathlib

In [35]:
# change this line to suit your needs
rwrd = "/home/nguyenah1112/networkTALL/misc_code/RWR"

# input network file
network_file = os.path.join(rwrd, "data/PPI_HiUnion_LitBM_APID_gene_names_190123_nodes_only.tsv")

# input weights file: weigths are either binary (0 or 1) or "-log(p-value)" to have low p-value -> big number and high p-value -> low number
weights_file = os.path.join(rwrd, "data/HOXA_tAML_padj_neglog_RWR.tsv")

# output file 
outfile = os.path.join(rwrd, "output/RW_HOXA_tAML.tsv")

In [36]:
# should be a 2-column CSV of edge list. And edge is a pair of nodes.
g = gt.load_graph_from_csv(network_file, csv_options={'delimiter': '\t'})

In [37]:
g.num_vertices()

15390

In [38]:
# create dict, to be able to get node object from node name
name2node = {}
for v in g.iter_vertices():
    name2node[g.vp.name[v]] = v

In [39]:
# read weights file and add to graphs as pers property (personalized)
with open(weights_file, "r") as f:
    csv_reader = csv.reader(f, delimiter='\t')
    list_of_rows = list(csv_reader)

In [41]:
list_of_rows[0]

['VPREB3', '0.003207879130690995']

In [30]:
g.vp.pers = g.new_vertex_property('double')
for gene_name, norm_DE in list_of_rows:
    n = name2node[gene_name]
    g.vp.pers[n] = norm_DE

KeyError: 'gene_name'

In [31]:
# returns the distribution vector, and the number of iterations done
v_distrib = gt.graph_tool.centrality.pagerank(g,
                        pers=g.vp.pers,
                        damping=0.5,
                        epsilon=1e-09,
                        max_iter=g.num_edges())

In [32]:
# adds the vector distribution to the graph (so that if we save the graph, the node values will be saved)
g.vp.v_distrib = v_distrib

In [33]:
# save the vector distribution to file
with open(outfile, "w") as f:
    for v in g.iter_vertices():
        print(f"{g.vp.name[v]}\t{g.vp.v_distrib[v]:.9f}", file=f)